This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [38]:
# First, import the relevant modules
import requests
import json
import os
import statistics

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [49]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

base_nasdaq_url = f'https://data.nasdaq.com/api/v3/datasets/'
database_code = 'FSE/'
dataset_code = 'AFX_X/'
return_format = 'json'
start_date='2020-12-01'
end_date='2020-12-01'

# grab one day of data
url = f'{base_nasdaq_url}{database_code}{dataset_code}data.{return_format}?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'
response = requests.get(url)

In [50]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response_json = response.json()
column_names =  response_json["dataset_data"]["column_names"]
column_names = [name.lower() for name in column_names]
open_index = column_names.index('open')
close_index = column_names.index('close')
high_index = column_names.index('high')
low_index = column_names.index('low')
change_index = column_names.index('change')
traded_volume_index = column_names.index('traded volume')

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-12-01', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [51]:
start_date='2017-01-01'
end_date='2017-12-31'
url = f'{base_nasdaq_url}{database_code}{dataset_code}data.{return_format}?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'
response = requests.get(url)

### 2. Convert the returned JSON object into a Python dictionary.

In [52]:
response_json = response.json()

### 3. Calculate the highest and lowest opening prices for the stock in this period.

In [53]:
data = response_json['dataset_data']['data']
max_open = max([daily_data[open_index] for daily_data in data if daily_data[open_index]])
min_open = min([daily_data[open_index] for daily_data in data if daily_data[open_index]])
print(f'\n High opening price of AFX_X in 2017: {max_open:0.3f}\n Low opening price of AFX_X in 2017: {min_open:0.3f}\n' )



 High opening price of AFX_X in 2017: 53.110
 Low opening price of AFX_X in 2017: 34.000



### 4. What was the largest change in any one day (based on High and Low price)?

In [54]:
max_single_day_change = max([daily_data[high_index] - daily_data[low_index] for daily_data in data\
                                              if (daily_data[high_index] and daily_data[low_index])])
print(f'\n Largest single day change for AFX_X in 2017: {max_single_day_change:0.3f}\n')


 Largest single day change for AFX_X in 2017: 2.810



### 5. What was the largest change between any two days (based on Closing Price)?

In [55]:
consecutive_day_changes = [data[i + 1][close_index] - data[i][close_index]  for i in range(len(data) - 1)\
                                              if (data[i + 1][close_index] and data[i][close_index])]
max_drop = min(consecutive_day_changes)
max_jump = max(consecutive_day_changes)
max_change = max_jump
if abs(max_drop) > max_jump:
    max_change = max_drop

print(f'\n Largest change in closing price between any two consecutive days of AFX_X in 2017: {max_change:0.3f} \n')


 Largest change in closing price between any two consecutive days of AFX_X in 2017: 2.560 



### 6. What was the average daily trading volume during this year?

In [56]:
average_daily_reading_volume = statistics.mean([daily_data[traded_volume_index] for\
                                                daily_data in data if daily_data[traded_volume_index]])
print(f'\n Average daily trading volume of AFX_X during 2017: {average_daily_reading_volume:0.1f}\n')


 Average daily trading volume of AFX_X during 2017: 89124.3



### 7a. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [57]:
median_daily_reading_volume = statistics.median([daily_data[traded_volume_index] for\
                                                daily_data in data if daily_data[traded_volume_index]])
print(f'\n Median daily trading volume of AFX_X during 2017: {median_daily_reading_volume:0.1f}\n')


 Median daily trading volume of AFX_X during 2017: 76286.0



### 7b. Alternate implementation

In [58]:
def median(lst = []):
    if not lst or not len(lst):
        return None
    
    list_len = len(lst)
    if list_len % 2 == 1:
        idx = list_len // 2
        median = lst[idx]
    else:
        high_idx = list_len // 2
        low_idx = high_idx - 1
        median = (lst[high_idx] + lst[low_idx])/2.
    return median

daily_trading_volumes = [daily_data[traded_volume_index] for\
                                                daily_data in data\
                         if daily_data[traded_volume_index]]
daily_trading_volumes.sort()
median_daily_reading_volume = median(daily_trading_volumes)
print(f'\n Median daily trading volume of AFX_X during 2017: {median_daily_reading_volume:0.1f}\n')


 Median daily trading volume of AFX_X during 2017: 76286.0

